<a href="https://colab.research.google.com/github/annisaeka123/Machine_Learning-Semester5/blob/main/JS06/Praktikum6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Praktikum 6

In [1]:
!pip install faiss-cpu
!pip install annoy
!pip install hnswlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp312-cp312-linux_x86_64.whl size=551809 sha256=4a0a783d9c0a2fcdb315201828a105c6491f520b241a9a74668a4c26a58643b6
  Stored in directory: /root/.cache/pip/wheels/db/b9/53/a3b2d1fe1743abadddec6aa541294b24fdbc39d7800bc57311
Successfully built annoy
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hnswlib: filename=hnswlib-0.8.0-cp312-cp312-linux_x86_64.whl size=2528145 sha256=ffaa93d2725977dc1bdf95aa63752a63f08afe46088b18752d065eb0d3b8d503
  Stored in directory: /root/.cache/pip/wheels/ac/39/b3/cbd7f9cbb76501d2d5fbc84956e70d0b94e788aac87bda465e
Successfully built hnswlib


In [2]:
import pandas as pd
import numpy as np
import time
import faiss
from annoy import AnnoyIndex
import hnswlib
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler


Load Dataset dan Ambil Fitur Numerik

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/Pembelajaran Mesin/spotify_songs.csv')
features = ['danceability', 'energy', 'loudness', 'speechiness',
            'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
X = df[features].values

Normalisasi Data

In [6]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


Exact Nearest Neighbor (Baseline)

In [7]:
# Jumlah tetangga terdekat yang akan dicari
k = 10

In [8]:
start = time.time()
nn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric='euclidean')
nn.fit(X_scaled)
dist_exact, idx_exact = nn.kneighbors(X_scaled)
time_exact = time.time() - start
print(f"Exact NN done in {time_exact:.3f} s")

Exact NN done in 5625.488 s


Annoy (Approximate Nearest Neighbors Oh Yeah)

In [9]:
start = time.time()
f = X_scaled.shape[1]
index_annoy = AnnoyIndex(f, 'euclidean')
for i, v in enumerate(X_scaled):
    index_annoy.add_item(i, v)
index_annoy.build(10)
idx_annoy = [index_annoy.get_nns_by_vector(v, k) for v in X_scaled]
time_annoy = time.time() - start
print(f"Annoy done in {time_annoy:.3f} s")


Annoy done in 71.201 s


HNSW (Hierarchical Navigable Small World)

In [10]:
start = time.time()
p_hnsw = hnswlib.Index(space='l2', dim=X_scaled.shape[1])
p_hnsw.init_index(max_elements=X_scaled.shape[0], ef_construction=200, M=16)
p_hnsw.add_items(X_scaled)
p_hnsw.set_ef(200)
idx_hnsw, dist_hnsw = p_hnsw.knn_query(X_scaled, k=k)
time_hnsw = time.time() - start
print(f"HNSW done in {time_hnsw:.3f} s")

HNSW done in 354.965 s


FAISS (Facebook AI Similarity Search) IVF

In [11]:
X_scaled = X_scaled.astype('float32')

start = time.time()
quantizer = faiss.IndexFlatL2(X_scaled.shape[1])
index_faiss = faiss.IndexIVFFlat(quantizer, X_scaled.shape[1], 100, faiss.METRIC_L2)
index_faiss.train(X_scaled)
index_faiss.add(X_scaled)
index_faiss.nprobe = 10
dist_faiss, idx_faiss = index_faiss.search(X_scaled, k)
time_faiss = time.time() - start
print(f"FAISS IVF done in {time_faiss:.3f} s")

FAISS IVF done in 746.286 s


Perbandingan Hasil dan Waktu

In [12]:
print(f"Exact NN done in {time_exact:.3f} s")
print(f"Annoy done in {time_annoy:.3f} s")
print(f"HNSW done in {time_hnsw:.3f} s")
print(f"FAISS IVF done in {time_faiss:.3f} s")

print("\nTop-5 neighbors for first song:")
print(f"Exact NN: {idx_exact[0][:5]}")
print(f"Annoy:    {idx_annoy[0][:5]}")
print(f"HNSW:     {idx_hnsw[0][:5]}")
print(f"FAISS:    {idx_faiss[0][:5]}")


Exact NN done in 5625.488 s
Annoy done in 71.201 s
HNSW done in 354.965 s
FAISS IVF done in 746.286 s

Top-5 neighbors for first song:
Exact NN: [     0 394553 764272 837727 749223]
Annoy:    [0, 764272, 837727, 749223, 61511]
HNSW:     [     0 394553 764272 837727 749223]
FAISS:    [     0 394553 764272 837727 749223]


Berdasarkan hasil eksekusi, Exact NN paling akurat namun paling lambat (5625,488 detik), sedangkan Annoy tercepat (71,201 detik) dengan sedikit perbedaan hasil. HNSW (354,965 detik) dan FAISS IVF (746,286 detik) menawarkan keseimbangan antara kecepatan dan akurasi, karena keduanya mampu menemukan tetangga yang sama dengan Exact NN. Secara keseluruhan, pilihan algoritma tergantung prioritas: akurasi penuh menggunakan Exact NN, kecepatan dengan Annoy, dan kompromi antara keduanya dengan HNSW atau FAISS IVF.